In [1]:
import sys
import copy
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split

In [2]:
def count(my_array):
    # Initialize the count
    norm_count = 0
    ano_count = 0

    # Iterate through each element in the list
    for id in my_array:
        if 'normal' in id:
            norm_count += 1
        elif 'anomaly' in id:
            ano_count += 1

    return (ano_count, norm_count)

In [3]:
# n_features = 7  # Number of features
# sequence_length = 20  # Length of each sequence

df = pd.read_csv('data.csv')
video_ids = df['video_id'].unique()

# Split video IDs into train, validation, and test sets
train_ids, temp_ids = train_test_split(video_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

train_ano_norm = count(train_ids)
val_ano_norm = count(val_ids)
test_ano_norm = count(test_ids)

print('training (anomaly, normal): ', train_ano_norm)
print('validation (anomaly, normal): ', val_ano_norm)
print('testing (anomaly, normal): ', test_ano_norm)


training (anomaly, normal):  (47, 51)
validation (anomaly, normal):  (13, 8)
testing (anomaly, normal):  (10, 11)


In [8]:
X_train = []
y_train = []

X_val = []
y_val = []

X_test = []
y_test = []


# Process train, validation, and test sets
for test_id in test_ids[:2]:
    filtered_df = df[df['video_id'] == test_id]
    features = filtered_df[['frame_num','crowd_density', 'loitering', 'no_concealment', 'low_concealment', 'med_concealment', 'high_concealment']]
    RBP = filtered_df['rbp'].iloc[0]
    features = features.to_numpy()

    # Reshape features into a single sequence
    # features = torch.tensor(features).unsqueeze(0)  # Add batch dimension

    # Pad the sequence
    # padded_features = pad_sequence(features, batch_first=True)

    # Append to X_train
    X_test.extend(features)
    X_test = np.array(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'extend'

In [5]:
X_test

[tensor([[[  1.0000,   0.0000,  -1.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  2.0000,   2.5245,   0.0000,  ...,   0.0000,   0.0000,   1.0000],
          [  3.0000,   2.6224,   0.0000,  ...,   0.0000,   0.0000,   1.0000],
          ...,
          [314.0000,   6.9848, 124.5427,  ...,   1.0000,   0.0000,   1.0000],
          [315.0000,   6.9296, 124.9462,  ...,   1.0000,   0.0000,   1.0000],
          [316.0000,   7.0662, 125.3502,  ...,   1.0000,   0.0000,   1.0000]]],
        dtype=torch.float64),
 tensor([[[  1.0000,   0.0000,  -1.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  2.0000,   0.0000,  -1.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  3.0000,   0.0000,  -1.0000,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [249.0000,   0.3409,   8.0000,  ...,   0.0000,   0.0000,   1.0000],
          [250.0000,   0.3436,   8.5000,  ...,   0.0000,   0.0000,   1.0000],
          [251.0000,   0.3482,   9.0000,  ...,   0.0000,   0.0000,   1.0000]]],


In [ ]:

for train_id in train_ids:
    filtered_df = df[df['video_id'] == train_id]
    features = filtered_df[['crowd_density', 'loitering', 'no_concealment', 'low_concealment', 'med_concealment', 'high_concealment']]
    RBP = filtered_df['rbp'].iloc[0]
    features = features.to_numpy()
    # n_seq = len(features) // sequence_length

    # Reshape the features into batches
    features = features[:sequence_length * n_seq].reshape(-1, sequence_length, n_features) # one video is one batch
    labels = [RBP] * n_seq

    # Extend the lists
    X_train.extend(features)
    y_train.extend(labels)


In [ ]:


for val_id in val_ids:
    filtered_df = df[df['video_id'] == val_id]
    features = filtered_df[['crowd_density', 'loitering', 'no_concealment', 'low_concealment', 'med_concealment', 'high_concealment']]
    RBP = filtered_df['rbp'].iloc[0]
    features = features.to_numpy()
    n_seq = len(features) // sequence_length

    # Reshape the features into batches
    features = features[:sequence_length * n_seq].reshape(-1, sequence_length, n_features) # one video is one batch
    labels = [RBP] * n_seq

    # Extend the lists
    X_val.extend(features)
    y_val.extend(labels)

for test_id in test_ids:
    filtered_df = df[df['video_id'] == test_id]
    features = filtered_df[['crowd_density', 'loitering', 'no_concealment', 'low_concealment', 'med_concealment', 'high_concealment']]
    RBP = filtered_df['rbp'].iloc[0]
    features = features.to_numpy()
    n_seq = len(features) // sequence_length

    # Reshape the features into batches
    features = features[:sequence_length * n_seq].reshape(-1, sequence_length, n_features) # one video is one batch
    labels = [RBP] * n_seq

    # Extend the lists
    X_test.extend(features)
    y_test.extend(labels)

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, 1)  # Single output neuron for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.fc(out[:, -1])  # Use the last hidden state for prediction (summary)
        out = self.sigmoid(out) # Apply sigmoid activation
        return out

In [ ]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=False, stateful=True)
#         self.fc = nn.Linear(hidden_size, 1)  # Single output neuron for binary classification
#         self.sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification

#     def forward(self, x):
#         out, _ = self.lstm(x.unsqueeze(0))  # Add batch dimension (batch_size=1)
#         out = self.fc(out[:, -1, :])  # Use the last hidden state for prediction
#         out = self.sigmoid(out)  # Apply sigmoid activation
#         return out

In [ ]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size):
#         super(LSTMModel, self).__init__()
#         self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
#         self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)  # Additional LSTM layer
#         self.fc1 = nn.Linear(hidden_size, 64)  # First dense layer
#         self.fc2 = nn.Linear(64, 32)  # Second dense layer
#         self.fc3 = nn.Linear(32, 1)  # Single output neuron
#         self.fc4 = nn.Sigmoid()  # Sigmoid activation for binary classification

#     def forward(self, x):
#         out, (h_n, c_n) = self.lstm1(x)
#         out, _ = self.lstm2(out, (h_n, c_n))  # Pass through the second LSTM layer
#         out = self.fc1(out[:, -1, :])
#         out = self.fc2(out)
#         out = self.fc3(out)
#         out = self.fc4(out)
#         return out

In [ ]:
# Instantiate the model
input_size = n_features
hidden_size = 64
model = LSTMModel(input_size, hidden_size)

# Define loss function and optimizer
# criterion = nn.MSELoss()-------------------------------------------
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Initialize empty lists to store training and validation losses
train_losses = []
val_losses = []
val_loss_temp = float('inf')

# Training loop
n_epochs = 300
patience = og_patience = 50
for epoch in range(1, n_epochs+1):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.unsqueeze(1))
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor.unsqueeze(1))

    # Append losses to lists
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    # save best model
    if val_loss.item() < val_loss_temp:
        best_model, best_epoch = copy.deepcopy(model), epoch
        val_loss_temp = val_loss.item()
        patience = 51

    patience -= 1
    if patience == 0:
        print(f"\nNo progress in the last {og_patience} epochs. Training completed.")
        break

    # print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    # Training progress bar
    label = f"{epoch}/{n_epochs} epochs"
    progress = (epoch)/ n_epochs
    progress_bar_len = 50
    filled_len = int(progress_bar_len * progress)
    bar = '=' * filled_len + '-' * (progress_bar_len - filled_len)
    sys.stdout.write(f'\rTraining in progress: [{bar}] {progress * 100:.1f}% [{label}]')
    sys.stdout.flush()

# Plot the losses
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Losses")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
model = best_model
print(f"Best model at epoch {best_epoch}.")

In [ ]:
# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)

    test_loss = criterion(test_outputs, y_test_tensor.unsqueeze(1))
    print(f"Test Loss: {test_loss.item():.4f}")

In [ ]:
# Evaluation
precision, recall, thresholds = precision_recall_curve(y_test_tensor, test_outputs)

# Compute F1 score (avoid NaN)
f1_scores = np.divide(2 * precision * recall, precision + recall, out=np.zeros_like(precision), where=(precision + recall) != 0)

# Find the optimal threshold that maximizes the F1 score
optimal_threshold = thresholds[f1_scores.argmax()]
max_f1_score = f1_scores.max()

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Max F1 Score: {max_f1_score:.4f}")

# Plot the F1 curve and the vertical line for the optimal threshold
plt.figure(figsize=(12, 6))

# F1 curve
plt.subplot(1, 2, 1)
plt.plot(thresholds, f1_scores[:-1], label='F1 Score')
plt.axvline(x=optimal_threshold, color='r', linestyle='--', label='Optimal Threshold')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Curve')
plt.grid(True)
plt.legend()

# Confusion matrix
plt.subplot(1, 2, 2)
test_outputs = (test_outputs >= optimal_threshold).float()
cm = confusion_matrix(y_test_tensor, test_outputs)
labels = ['No Robbery', 'Robbery']
cm_transposed = cm.T

sns.heatmap(cm_transposed, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('True Labels')
plt.ylabel('Predicted Labels')

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

In [ ]:
# Save the model
# torch.save(model.state_dict(), f'model_{optimal_threshold:.4f}.pt')

In [ ]:
for i in range(int(len(features))):

    input_data = X_train[i]

    # Preprocess data
    input_data = scaler.fit_transform(input_data.reshape(-1, n_features)).reshape(-1, sequence_length, n_features)[0]
    input_data = input_data.tolist()
    input_data = torch.tensor(input_data, dtype=torch.float32)

    # Ensure input shape matches
    if input_data.shape != (sequence_length, n_features):
        print('error: ', f'Input shape must be {(sequence_length, n_features)}. Yours has shape {input_data.shape}')

    # Make predictions
    with torch.no_grad():
        output = model(input_data.unsqueeze(0))  # Add batch dimension
        RBP = (output).squeeze().cpu().numpy()

    actualRBP = y_train[i]

    print(f"\nprediction: {RBP:.2f} | actual: {actualRBP}", end=" ")
    if ((RBP >= optimal_threshold) & (actualRBP == 1)) | ((RBP < optimal_threshold) & (actualRBP == 0)):
        print("| CORRECT", end="")

In [ ]:
# IF NEEDED

# # Plot the precision-recall curve
# plt.figure(figsize=(8, 6))
# plt.plot(recall, precision, label='Precision-Recall Curve')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Precision-Recall Curve')
# plt.grid(True)
# plt.legend()
# plt.show()

# # Plot the F1 curve
# plt.figure(figsize=(8, 6))
# plt.plot(thresholds, f1_scores[:-1], label='F1 Score')
# plt.axvline(x=optimal_threshold, color='r', linestyle='--', label='Optimal Threshold')
# plt.xlabel('Threshold')
# plt.ylabel('F1 Score')
# plt.title('F1 Curve')
# plt.grid(True)
# plt.legend()
# plt.show()

# # Plot the Confusion matrix
# test_outputs = (test_outputs >= optimal_threshold).float()  # Convert probabilities to 0 or 1
# # test_outputs.squeeze(-1)

# cm = confusion_matrix(y_test_tensor, test_outputs)
# labels = ['No Robbery', 'Robbery']

# # Transpose the confusion matrix and swap axis labels
# cm_transposed = cm.T

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_transposed, annot=True, fmt='d', cmap='Blues', cbar=False,
#             xticklabels=labels, yticklabels=labels)  # Set labels for x and y axes
# plt.title('Confusion Matrix')
# plt.xlabel('True Labels')
# plt.ylabel('Predicted Labels')
# plt.show()
